In [50]:
import pandas as pd
import cv2
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score,f1_score,classification_report,confusion_matrix
from sklearn.preprocessing import LabelEncoder
import numpy as np
import os
image_count = {}
default_image_size = tuple((128, 128))

data = []
c=-1
for root, _, files in os.walk('/kaggle/input/plantdisease/PlantVillage'):
    disease = os.path.basename(root)
    print(disease)
    image_count[disease] = 0  
    c+=1
#     if((c>=3 and c<=10) or c==13 or c==15):
#     if(c==2 or c==12 or c==14):
    if(c==1 or c==11):
        for file in files:
            if file.endswith('.jpg') or file.endswith('.png') or file.endswith('.JPG') or file.endswith('.PNG') or file.endswith('JPEG') or file.endswith('jpeg') :
                image_path = os.path.join(root, file)

                if image_count[disease] >= 1000:
                    continue

                # Read the image
                image = cv2.imread(image_path)
    #             image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
                image = cv2.resize(image, default_image_size)
                # Flatten the image to a 1D array and convert to NumPy array
                flattened_image = image.flatten().astype(int)
                # Store the flattened pixel values and disease name in data
                data.append([flattened_image, disease])

                # Increment the counter for the current disease
                image_count[disease] += 1

df = pd.DataFrame(data, columns=['image_pixels', 'disease'])

# Split the data into training and testing sets
X = np.vstack(df['image_pixels'].to_numpy())
y = df['disease']

# Encode disease labels using LabelEncoder
le = LabelEncoder()
y = le.fit_transform(y)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


# Train the model
model = RandomForestClassifier(n_estimators=100,random_state=42)
model.fit(X_train, y_train)

# Make predictions
y_pred = model.predict(X_test)

# Evaluate the model






PlantVillage
Pepper__bell___Bacterial_spot
Potato___healthy
Tomato_Leaf_Mold
Tomato__Tomato_YellowLeaf__Curl_Virus
Tomato_Bacterial_spot
Tomato_Septoria_leaf_spot
Tomato_healthy
Tomato_Spider_mites_Two_spotted_spider_mite
Tomato_Early_blight
Tomato__Target_Spot
Pepper__bell___healthy
Potato___Late_blight
Tomato_Late_blight
Potato___Early_blight
Tomato__Tomato_mosaic_virus


In [51]:
# print(X_train)
# print(y_train)
# print(X_train.shape)


In [52]:
len(X_train[0])

49152

In [53]:
print(image_count)

{'PlantVillage': 0, 'Pepper__bell___Bacterial_spot': 997, 'Potato___healthy': 0, 'Tomato_Leaf_Mold': 0, 'Tomato__Tomato_YellowLeaf__Curl_Virus': 0, 'Tomato_Bacterial_spot': 0, 'Tomato_Septoria_leaf_spot': 0, 'Tomato_healthy': 0, 'Tomato_Spider_mites_Two_spotted_spider_mite': 0, 'Tomato_Early_blight': 0, 'Tomato__Target_Spot': 0, 'Pepper__bell___healthy': 1000, 'Potato___Late_blight': 0, 'Tomato_Late_blight': 0, 'Potato___Early_blight': 0, 'Tomato__Tomato_mosaic_virus': 0}


In [54]:
accuracy = accuracy_score(y_test, y_pred)
print('Accuracy:', accuracy)

Accuracy: 0.8825


In [55]:
f1 = f1_score(y_test, y_pred, average='weighted')
f1

0.8822972549019608

In [56]:
conf_matrix = confusion_matrix(y_test, y_pred)
conf_matrix

array([[189,  18],
       [ 29, 164]])